In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [ ]:
# Connecting to PostgreSQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Build a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Extract Video Game reviews from S3 bucket
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True )

# Show DataFrame
video_games_df.show()

In [ ]:
# Count the number of rows in video_games_df
record_count = video_games_df.count()

print(f'There are {record_count} records in this dataframe')


In [ ]:
# Transform video_games_df into the review_id_table schema in resources
from pyspark.sql.types import DateType

# Create a list of the columns we want
review_id_columns = ['review_id' , 'customer_id' , 'product_id' , 'product_parent' , 'review_date']

# Subset video_games_df using the list
review_id_df = video_games_df[review_id_columns]

review_id_df.show()

# Check that the dtypes match what is requested in the schema
# review_date is string , that needs to change

# Convert review_date to date type
review_id_df = review_id_df.withColumn('review_date' , review_id_df['review_date'].cast(DateType()))

# confirm type conversion
review_id_df.dtypes





In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c0iyntsfllgk.us-west-1.rds.amazonaws.com:5432/HW22"
config = {"user":"root", 
          "password": "Password", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# writing to review_id_table in PGadmin
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Creating df to match products schema

# Creating a list of the columns needed
productList = ['product_id' , 'product_title']

# Creating a subset from video_games_df
products_df = video_games_df[[productList]]

products_df.show()


In [ ]:
# Writing products_df to the products schema
review_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# creating a df to match the customers schema
from pyspark.sql.functions import lit

# Creating subset of video_games_df
customers_df = video_games_df[customersList]


# creating the customer count column
customers_df = customers_df.withColumn('customer_count' ,lit(record_count) )
customers_df.show()

In [ ]:
# writing customers_df to the customers schema
review_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# creating a df to match the vine schema

# listing the needed columns
vineColumns = ['review_id' , 'star_rating' , 'helpful_votes' , 'total_votes' , 'vine']

# creating a subset of video_games_df
vine_df = video_games_df[vineColumns]
vine_df.show()

In [ ]:
# writing to the vine_table using vine_df
review_id_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)